# Import Statements

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv1D, GlobalAveragePooling1D, Dense, Dropout, Input
import os

2025-07-07 11:12:05.734646: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-07 11:12:05.737858: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-07 11:12:05.746479: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751886725.761980   12005 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751886725.768965   12005 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1751886725.787172   12005 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [2]:
print(tf.__version__)

2.19.0


# Generate synthetic data and save as CSV if not present

In [3]:
csv_path = "health_data.csv"
if not os.path.exists(csv_path):
    num_days = 120
    np.random.seed(42)
    dates = pd.date_range(end=pd.Timestamp.today(), periods=num_days)
    df_synth = pd.DataFrame({
        'date': dates,
        'HRV': np.random.normal(50, 10, num_days),
        'RHR': np.random.normal(60, 5, num_days),
        'Sleep': np.random.normal(7, 1, num_days),
        'Steps': np.random.normal(8000, 2000, num_days),
        'Mood': np.random.normal(3, 1, num_days),
        'Readiness': np.random.normal(75, 10, num_days)
    })
    df_synth.to_csv(csv_path, index=False)
    print(f"Synthetic data generated and saved to {csv_path}")

# Load and preprocess data

In [4]:
df = pd.read_csv(csv_path, parse_dates=['date'])
df = df.sort_values('date')

feature_cols = ['HRV', 'RHR', 'Sleep', 'Steps', 'Mood']
target_col = 'Readiness'

df[feature_cols] = (df[feature_cols] - df[feature_cols].mean()) / df[feature_cols].std()

# Create sequences for model input

In [5]:
sequence_length = 7
X = []
y = []

for i in range(len(df)-sequence_length):
    seq = df[feature_cols].iloc[i:i+sequence_length].values
    label = df[target_col].iloc[i+sequence_length]
    X.append(seq)
    y.append(label)

X = np.array(X)
y = np.array(y)

print(f"Shape of input X: {X.shape}")
print(f"Shape of labels y: {y.shape}")
    

Shape of input X: (113, 7, 5)
Shape of labels y: (113,)


# 3. Build the Conv1D model (TFLite-compatible)

In [6]:
def build_readiness_model(seq_len, num_features):
    model = Sequential([
        Input(shape=(seq_len, num_features), name="input"),
        Conv1D(32, kernel_size=3, activation='relu', padding='same'),
        GlobalAveragePooling1D(),
        Dense(32, activation='relu'),
        Dropout(0.2),
        Dense(1, activation='linear')
    ])
    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    return model

model = build_readiness_model(sequence_length, len(feature_cols))
model.summary()

2025-07-07 11:12:08.705187: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv1d (Conv1D)                 │ (None, 7, 32)          │           512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling1d        │ (None, 32)             │             0 │
│ (GlobalAveragePooling1D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 32)             │         1,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,601 (6.25 KB)

 Trainable params: 1,601 (6.25 KB)

 Non-trainable params: 0 (0.00 B)

# Train the model

In [7]:
history = model.fit(
    X, y,
    epochs=30,
    batch_size=8,
    validation_split=0.2,
    verbose=2
)

Epoch 1/30


12/12 - 1s - 85ms/step - loss: 5686.0278 - mae: 74.6514 - val_loss: 5522.0845 - val_mae: 73.6440
Epoch 2/30
12/12 - 0s - 6ms/step - loss: 5631.1523 - mae: 74.2836 - val_loss: 5465.8525 - val_mae: 73.2625
Epoch 3/30
12/12 - 0s - 6ms/step - loss: 5564.4116 - mae: 73.8292 - val_loss: 5399.3804 - val_mae: 72.8086
Epoch 4/30
12/12 - 0s - 6ms/step - loss: 5488.0439 - mae: 73.3075 - val_loss: 5322.1489 - val_mae: 72.2778
Epoch 5/30
12/12 - 0s - 6ms/step - loss: 5397.9395 - mae: 72.7005 - val_loss: 5231.9048 - val_mae: 71.6524
Epoch 6/30
12/12 - 0s - 6ms/step - loss: 5264.8130 - mae: 71.7810 - val_loss: 5123.3237 - val_mae: 70.8922
Epoch 7/30
12/12 - 0s - 5ms/step - loss: 5125.4336 - mae: 70.8073 - val_loss: 4992.5430 - val_mae: 69.9655
Epoch 8/30
12/12 - 0s - 6ms/step - loss: 4987.4482 - mae: 69.8194 - val_loss: 4838.7842 - val_mae: 68.8603
Epoch 9/30
12/12 - 0s - 6ms/step - loss: 4754.8618 - mae: 68.1636 - val_loss: 4659.3223 - val_mae: 67.5465
Epoch 10/30
12/12 - 0s - 6ms/step - loss: 4545.

# Export to Tensorflow Lite model

In [8]:
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

with open("readiness_model.tflite", "wb") as f:
    f.write(tflite_model)

print("TFLite model saved as readiness_model.tflite")

INFO:tensorflow:Assets written to: /tmp/tmpsdlexe6z/assets


INFO:tensorflow:Assets written to: /tmp/tmpsdlexe6z/assets


Saved artifact at '/tmp/tmpsdlexe6z'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 7, 5), dtype=tf.float32, name='input')
Output Type:
  TensorSpec(shape=(None, 1), dtype=tf.float32, name=None)
Captures:
  135760500637328: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135760500636560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135760495002448: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135760495004560: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135760495003792: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135760495005136: TensorSpec(shape=(), dtype=tf.resource, name=None)
TFLite model saved as readiness_model.tflite


W0000 00:00:1751886732.382636   12005 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1751886732.382659   12005 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
2025-07-07 11:12:12.382969: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpsdlexe6z
2025-07-07 11:12:12.383363: I tensorflow/cc/saved_model/reader.cc:52] Reading meta graph with tags { serve }
2025-07-07 11:12:12.383371: I tensorflow/cc/saved_model/reader.cc:147] Reading SavedModel debug info (if present) from: /tmp/tmpsdlexe6z
I0000 00:00:1751886732.386731   12005 mlir_graph_optimization_pass.cc:425] MLIR V1 optimization pass is not enabled
2025-07-07 11:12:12.387417: I tensorflow/cc/saved_model/loader.cc:236] Restoring SavedModel bundle.
2025-07-07 11:12:12.407982: I tensorflow/cc/saved_model/loader.cc:220] Running initialization op on SavedModel bundle at path: /tmp/tmpsdlexe6z
2025-07-07 11:12:12.414342: I tensorflow/cc/saved_model/loader.cc:471] SavedModel 